In [39]:
import pandas as pd 
import cleaning
from collections import defaultdict
diabetes = pd.read_csv("./data/diabetic_data.csv")
ccs_mapping = pd.read_csv("./data/ccs_multi_dx_tool_2015.csv", dtype = str)
diabetes = cleaning.load_and_clean(diabetes)

print(ccs_mapping.head())

  'ICD-9-CM CODE' 'CCS LVL 1'                  'CCS LVL 1 LABEL' 'CCS LVL 2'  \
0         '01000'         '1'  Infectious and parasitic diseases       '1.1'   
1         '01001'         '1'  Infectious and parasitic diseases       '1.1'   
2         '01002'         '1'  Infectious and parasitic diseases       '1.1'   
3         '01003'         '1'  Infectious and parasitic diseases       '1.1'   
4         '01004'         '1'  Infectious and parasitic diseases       '1.1'   

     'CCS LVL 2 LABEL' 'CCS LVL 3'  'CCS LVL 3 LABEL' 'CCS LVL 4'  \
0  Bacterial infection     '1.1.1'  Tuberculosis [1.]         ' '   
1  Bacterial infection     '1.1.1'  Tuberculosis [1.]         ' '   
2  Bacterial infection     '1.1.1'  Tuberculosis [1.]         ' '   
3  Bacterial infection     '1.1.1'  Tuberculosis [1.]         ' '   
4  Bacterial infection     '1.1.1'  Tuberculosis [1.]         ' '   

  'CCS LVL 4 LABEL'  
0                    
1                    
2                    
3               

In [40]:
print(df['diag_1'].head())

0    250.83
1       276
2       648
3         8
4       197
Name: diag_1, dtype: object


In [ ]:
df = diabetes

ccs_mapping = pd.read_csv("./data/ccs_multi_dx_tool_2015.csv", dtype = str)
# Clean CCS mapping column names and values
ccs_mapping.columns = ccs_mapping.columns.str.strip("'").str.strip()
ccs_mapping = ccs_mapping.rename(columns={
    "ICD-9-CM CODE": "icd9_code",
    "CCS LVL 1": "ccs_level_1",
    "CCS LVL 1 LABEL": "ccs_description"
})
ccs_mapping["icd9_code"] = ccs_mapping["icd9_code"].str.strip("'").str.strip()
# Step 1: Remove dots from ICD-9 codes
for col in ["diag_1", "diag_2", "diag_3"]:
    df[f"{col}_clean"] = df[col].str.replace(".", "", regex=False)

# Step 2: Build prefix-based lookup table for fallback matching
prefix_map = defaultdict(lambda: None)
for code, desc in zip(ccs_mapping["icd9_code"], ccs_mapping["ccs_description"]):
    for i in range(1, len(code) + 1):  # Use prefixes starting at 3 digits
        prefix = code[:i]
        if prefix not in prefix_map:
            prefix_map[prefix] = desc

# Step 3: Matching function using longest prefix available
def map_icd_to_ccs(code):
    if pd.isna(code):
        return None
    for i in range(len(code), 2, -1):
        prefix = code[:i]
        if prefix in prefix_map:
            return prefix_map[prefix]
    return None

# Step 4: Map each diagnosis column to CCS group using prefix match
for col in ["diag_1_clean", "diag_2_clean", "diag_3_clean"]:
    df[col.replace("_clean", "_ccs")] = df[col].map(map_icd_to_ccs)

In [46]:
print(df[['diag_1_clean', 'diag_1_ccs']].head())

  diag_1_clean                                         diag_1_ccs
0        25083  Endocrine; nutritional; and metabolic diseases...
1          276  Endocrine; nutritional; and metabolic diseases...
2          648                                     Mental Illness
3            8                                               None
4          197                                          Neoplasms
